In [31]:
# import  libraries
import pandas as pd
import numpy as np
from statsmodels.stats.proportion import proportions_ztest
import matplotlib.pyplot as plt
import seaborn as sns

# Set plot style 
sns.set(style="whitegrid")

In [3]:
file_path = 'data_files/ab_data.csv'

# Load the dataset
df = pd.read_csv(file_path)

print("Data loaded successfully!")
print(df.head())

Data loaded successfully!
   user_id                   timestamp      group landing_page  converted
0   851104  2017-01-21 22:11:48.556739    control     old_page          0
1   804228  2017-01-12 08:01:45.159739    control     old_page          0
2   661590  2017-01-11 16:55:06.154213  treatment     new_page          0
3   853541  2017-01-08 18:28:03.143765  treatment     new_page          0
4   864975  2017-01-21 01:52:26.210827    control     old_page          1


In [5]:
# info about dataset
print("\nDataset Information:")
df.info()


Dataset Information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294478 entries, 0 to 294477
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   user_id       294478 non-null  int64 
 1   timestamp     294478 non-null  object
 2   group         294478 non-null  object
 3   landing_page  294478 non-null  object
 4   converted     294478 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 11.2+ MB


In [7]:
# Check for any users who might be in both the control and treatment group
duplicate_users = df['user_id'].duplicated().sum()
print(f"\nNumber of duplicate user entries: {duplicate_users}")



Number of duplicate user entries: 3894


In [9]:
# Let's see how many unique users we have
print(f"Number of unique users: {df['user_id'].nunique()}")



Number of unique users: 290584


In [15]:
# Remove users who are in the 'treatment' group but saw the 'old_page', and vice-versa.
# These records are not useful for the test.
mismatched_rows = df[((df['group'] == 'treatment') & (df['landing_page'] == 'old_page')) | ((df['group'] == 'control') & (df['landing_page'] == 'new_page'))]
print(f"\nNumber of mismatched rows to remove: {len(mismatched_rows)}")




Number of mismatched rows to remove: 3893


In [17]:
# Remove the mismatched rows
df_cleaned = df.drop(mismatched_rows.index)

# Another check: Ensure each user ID is now unique after cleaning
print(f"Number of duplicate user_ids after cleaning mismatches: {df_cleaned['user_id'].duplicated().sum()}")


Number of duplicate user_ids after cleaning mismatches: 1


In [19]:

# Remove the remaining duplicates
df_cleaned = df_cleaned.drop_duplicates(subset='user_id', keep='first')

print(f"\nFinal cleaned dataset contains {df_cleaned.shape[0]} unique user sessions.")


Final cleaned dataset contains 290584 unique user sessions.


In [21]:
# Separate  data into control and treatment groups
control_group = df_cleaned[df_cleaned['group'] == 'control']
treatment_group = df_cleaned[df_cleaned['group'] == 'treatment']



In [23]:
# Calculate conversion rates
control_conversion_rate = control_group['converted'].mean()
treatment_conversion_rate = treatment_group['converted'].mean()

print(f"Control Group (Old Page) Conversion Rate: {control_conversion_rate:.4%}")
print(f"Treatment Group (New Page) Conversion Rate: {treatment_conversion_rate:.4%}")

Control Group (Old Page) Conversion Rate: 12.0386%
Treatment Group (New Page) Conversion Rate: 11.8808%


In [25]:
# Get the number of conversions for each group
control_conversions = control_group['converted'].sum()
treatment_conversions = treatment_group['converted'].sum()



In [27]:
# Get the total number of users in each group
n_control = len(control_group)
n_treatment = len(treatment_group)

# Combine these into arrays for the test
count = np.array([treatment_conversions, control_conversions])
nobs = np.array([n_treatment, n_control])


In [33]:

# Perform the Z-test
# 'alternative="larger"' tests the hypothesis that the first group's proportion is larger than the second's.
z_stat, p_value = proportions_ztest(count, nobs, alternative='larger')

print(f"Z-statistic: {z_stat:.4f}")
print(f"P-value: {p_value:.4f}")

# Set our significance level (alpha)
alpha = 0.05

# Interpret the result
if p_value < alpha:
    print("\nResult: Reject the null hypothesis.")
    print("Conclusion: The new page has a statistically significant higher conversion rate.")
else:
    print("\nResult: Fail to reject the null hypothesis.")
    print("Conclusion: We do not have enough evidence to say the new page is better than the old page.")

Z-statistic: -1.3109
P-value: 0.9051

Result: Fail to reject the null hypothesis.
Conclusion: We do not have enough evidence to say the new page is better than the old page.


***The result of this project was definitive: the new webpage design failed. The statistical analysis yielded a high p-value (0.91), meaning we could not prove that the new page was any better than the old one. This result prevented the company from making a useless and potentially costly change, demonstrating the value of data-driven decision-making over intuition.***